<a href="https://colab.research.google.com/github/ellaroberson/Statistics-and-Probability/blob/main/Ella_lab_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install wandb #is a tool for visualizing and tracking machine
#learning experiments.
!apt-get install poppler-utils #poppler-utils is a set of utilities for working
#with PDF files, such as extracting text, images
# and other information from PDF documents.
!pip install pdf2image #converts PDF files into images
!pip install flashtorch #provides tools for visualizing activations, gradients,
#and filters within deep neural networks, aiding in the interpretation
import requests # library is used for making HTTP requests in Python
from pdf2image import convert_from_path # convert pages of a PDF file into images.
import matplotlib.pyplot as plt # creating visualizations in Python
import numpy as np
import torch
import requests
from torchvision import * #This is a package in PyTorch that consists of popular
# datasets, model architectures,
#and common image transformations for computer vision tasks.
from torchvision.models import *
import wandb as wb

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#This line checks if a CUDA-enabled GPU is available
#If a GPU is available, it sets the device variable to "cuda:0",
#indicating that computations should be performed on the GPU.
#If no GPU is available, it sets device to "cpu", indicating that computations
#should be performed on the CPU.
def GPU(data):
    return torch.tensor(data, requires_grad=True, dtype=torch.float, device=device)

def GPU_data(data):
    return torch.tensor(data, requires_grad=False, dtype=torch.float, device=device)
#These functions help streamline the process of creating PyTorch tensors on the
# appropriate device (CPU or GPU) while also managing whether gradients need to
#be tracked for those tensors
def plot(x):
    fig, ax = plt.subplots()
    im = ax.imshow(x, cmap = 'gray')
    ax.axis('off')
    fig.set_size_inches(5, 5)
    plt.show()

def get_google_slide(url): # takes a URL of a Google Slides presentation as input
#and returns a URL that can be used to download the presentation as a PDF file.
    url_head = "https://docs.google.com/presentation/d/" #stores the beginning part of the URL
    url_body = url.split('/')[5] #This line splits the input URL by / and extracts
    # the part of the URL that contains the unique identifier for the Google Slides presentation.
    #This part of the URL is typically after the fifth / occurrence.
    page_id = url.split('.')[-1]
    return url_head + url_body + "/export/pdf?id=" + url_body + "&pageid=" + page_id
    #the function returns the generated URL.

def get_slides(url): #converts each page of the PDF into images from the previous url
    url = get_google_slide(url) #transforms to a pdf
    r = requests.get(url, allow_redirects=True) #It allows redirects
    open('file.pdf', 'wb').write(r.content) #writes the content of the HTTP response
     #(r.content) to the file.
    images = convert_from_path('file.pdf', 500) #The function takes two arguments:
    # the path to the PDF file ('file.pdf') and the resolution (500 DPI in this case).
    return images

def load(image, size=224): #image, which represents the input image to be
    """

    Args:
      image:
      size:

    Returns:

    """
#processed, and size, which specifies the desired size for the image after preprocessing
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]
    #These lines define the mean and standard deviation values for normalization.
    transform = transforms.Compose([ #creates a series of image transformations
        transforms.Resize(size),
        transforms.CenterCrop(size),
        transforms.ToTensor(),
        transforms.Normalize(means, stds)
    ])
    tensor = transform(image).unsqueeze(0).to(device)
    tensor.requires_grad = True
    return tensor



In [ ]:
labels = {int(key):value for (key, value) in requests.get('https://s3.amazonaws.com/mlpipes/pytorch-quick-start/labels.json').json().items()}
#This line makes an HTTP GET request to the specified URL, which contains a JSON file (labels.json). The JSON file likely contains a mapping of class indices to class labels for the model's output.
model = alexnet(weights='DEFAULT').to(device)
#This function call creates an instance of the AlexNet model.This method moves the model to the specified device (device). If device is set to a GPU, the model will be transferred to the GPU; otherwise, it will remain on the CPU.
model.eval(); #changes the behavior of certain layers, such as dropout and batch normalization layers, to ensure consistent behavior during inference.

In [ ]:
url = "https://docs.google.com/presentation/d/17Nxy2Wo0erk71fp4sCDQqHrHYkxwjdExjbosjoEewuU/edit#slide=id.p"

In [ ]:
images = [] #store the processed images

for image in get_slides(url):

    plot(image)

    images.append(load(image))

images = torch.vstack(images)

In [ ]:
images.shape
#you can understand the size and structure of the tensor

In [ ]:
model(images) #produces the model's predictions or outputs for the given batch of images

In [ ]:
y = model(images) #sets that equal to y

In [ ]:
y.shape #gives the y shape dimensions

In [ ]:
guesses = torch.argmax(y, 1).cpu().numpy()
#torch.argmax(y, 1) means y is likely the output of the neural network, and 1 indicates that the maximum value should be searched for along dimension 1
#cpu(): This method moves the tensor from the GPU (if it was computed on the GPU) to the CPU
#.numpy(): This method converts the tensor to a NumPy array

In [ ]:
for i in list(guesses): #This loop iterates over each element (i) in the guesses list.
    print(labels[i]) #prints the class label corresponding to the current predicted class index (i).

In [ ]:
Y = np.zeros(50,)
Y[25:] = 1

In [ ]:
Y

In [ ]:
# Y = np.zeros(100,)
# Y[50:] = 1

In [ ]:
Y

In [ ]:
X = y.detach().cpu().numpy()

In [ ]:
X.shape

In [ ]:
plt.plot(X[0],'.')

In [ ]:
plt.hist(X[0]) #enerates a histogram plot of the values in the first element (X[0]) of the array X using Matplotlib.

In [ ]:
X = GPU_data(X)
Y = GPU_data(Y)

In [ ]:
def softmax(x): #computes the softmax activation function for each row of a tensor x.
    s1 = torch.exp(x - torch.max(x,1)[0][:,None])
    s = s1 / s1.sum(1)[:,None] #This computes the sum of each row of the tensor s1.
    return s

In [ ]:
def cross_entropy(outputs, labels): #computes the cross-entropy loss between the predicted outputs and the true labels
    return -torch.sum(softmax(outputs).log()[range(outputs.size()[0]), labels.long()])/outputs.size()[0]

In [ ]:
def randn_trunc(s): #Truncated Normal Random Numbers
    mu = 0 #This sets the mean of the normal distribution to 0.
    sigma = 0.1 #sets the standard deviation of the normal distribution to 0.1.
    R = stats.truncnorm((-2*sigma - mu) / sigma, (2*sigma - mu) / sigma, loc=mu, scale=sigma)
    #line creates a truncated normal distribution using the truncnorm function from the stats module
    return R.rvs(s)

In [ ]:
def Truncated_Normal(size): #generates samples from a truncated normal distribution using the Box-Muller transform.

    u1 = torch.rand(size)*(1-np.exp(-2)) + np.exp(-2) #This line generates uniform random numbers (u1) in the range (0, 1) using torch.rand(size)
    u2 = torch.rand(size) #This line generates another set of uniform random numbers (u2) in the range (0, 1).
    z  = torch.sqrt(-2*torch.log(u1)) * torch.cos(2*np.pi*u2)
    #This line applies the Box-Muller transform to transform the generated uniform random numbers into samples (z) from a standard normal distribution (mean=0, std=1) with the specified size.

    return z

In [ ]:
def acc(out,y): #calculates the accuracy of a model's predictions given the predicted outputs (out) and the true labels (y).
    with torch.no_grad(): #saves memory and computational resources since gradients are not needed for calculating accuracy.
        return (torch.sum(torch.max(out,1)[1] == y).item())/y.shape[0]

In [ ]:
X.shape #defines dimensions

In [ ]:
def get_batch(mode): #used to retrieve a batch of data samples and their corresponding labels for either training or testing purposes.
    b = c.b #etrieves the batch size (b) from some external source or configuration object c.
    if mode == "train": #checks if the requested mode is for training.
        r = np.random.randint(X.shape[0]-b) #This random integer represents the starting index of the batch in the training dataset.
        x = X[r:r+b,:] #This line selects a batch of input data samples (x) from the training dataset (X) starting from index r and spanning b samples.
        y = Y[r:r+b]
    elif mode == "test": #This condition checks if the requested mode is for testing.
        r = np.random.randint(X_test.shape[0]-b)
        x = X_test[r:r+b,:]
        y = Y_test[r:r+b]
    return x,y

In [ ]:
def model(x,w): #represents a simple linear regression model

    return x@w[0] # This line performs matrix multiplication between the input features x and the weights w[0]. The @ operator is used in Python for matrix multiplication (dot product).

In [ ]:
def make_plots(): #create plots related to model training and performance evaluation.

    acc_train = acc(model(x,w),y)

    # xt,yt = get_batch('test')

    # acc_test = acc(model(xt,w),yt)

    wb.log({"acc_train": acc_train})
    #it logs the training accuracy value (acc_train) to the logging platform under the key "acc_train".

In [ ]:
wb.init(project="Linear_Model_Photo_1"); #sets up the experiment environment and connects it to the specified project
c = wb.config # assigns the configuration object of the experiment to the variable c.

c.h = 0.001 #ets the learning rate (h) in the experiment configuration to 0.001.
c.b = 32 #This sets the batch size (b) in the experiment configuration to 32.
c.epochs = 100000 #This sets the number of epochs (epochs) in the experiment configuration to 100000.

w = [GPU(Truncated_Normal((1000,2)))] #This suggests that the model has 1000 weights for each of the 2 features.

optimizer = torch.optim.Adam(w, lr=c.h) #This initializes an Adam optimizer
 #(torch.optim.Adam) with the model weights w and the learning rate specified in the experiment configuration (c.h).

for i in range(c.epochs): #Each epoch represents one pass through the entire training dataset.

    x,y = get_batch('train') #This line retrieves a batch of training data samples
     #(x) and their corresponding labels (y)

    loss = cross_entropy(softmax(model(x,w)),y) #This computes the loss for the current batch of training data

    optimizer.zero_grad() #This line zeroes out the gradients of all model parameters before the backward pass.
    loss.backward() #computes the gradients of the loss with respect to all model parameters using backpropagation.
    optimizer.step() #This updates the model parameters (weights) based on the computed gradients and the optimization algorithm

    wb.log({"loss": loss}) #This logs the value of the loss for the current iteration to the logging platform (Weights & Biases).

    make_plots() #MAKES PLOTS
